---
### Imports

In [2]:
import glob
import pandas as pd
import numpy as np

---
## Loading CitiBike Data

In [5]:
bike_csv = glob.glob(r"data/JC-[0-9][0-9][0-9][0-9][0-9][0-9]-citibike-tripdata.csv")
bike_data = []

for file in bike_csv:
    df_bike = pd.read_csv(file)
    bike_data.append(df_bike)


Combine all the CSV's into a single DataFrame to be analysed.

In [8]:
df_bike = pd.concat(bike_data, ignore_index=True)

---
### Checking Column Names and Data Types

Print column names to determine whether processing necessary.

In [12]:
print(df_bike.columns)

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender'],
      dtype='object')


The white space, and capitalised characters complicate preparation of data. In order to simplify data preparation the columns are then renamed.

Rename all the columns to remove capitalised characters and replace whitespace with underscores.

In [16]:
df_bike.columns = [x.replace(' ', '_').lower() for x in df_bike.columns]

Make the `bike_id` the index so that it can become the primary key.

In [19]:
df_bike['bike_id'] = df_bike.index

Print the columns to verify that the renaming of the columns was successful.

In [22]:
print(df_bike.columns)

Index(['trip_duration', 'start_time', 'stop_time', 'start_station_id',
       'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bike_id', 'user_type',
       'birth_year', 'gender'],
      dtype='object')


Check the datatypes assigned to each column.

In [25]:
print(df_bike.dtypes)

trip_duration                int64
start_time                  object
stop_time                   object
start_station_id             int64
start_station_name          object
start_station_latitude     float64
start_station_longitude    float64
end_station_id               int64
end_station_name            object
end_station_latitude       float64
end_station_longitude      float64
bike_id                      int64
user_type                   object
birth_year                 float64
gender                       int64
dtype: object


Convert the column data types to their appropriate values. Trip duration is changed to be `stop_time - start_time`.

In [28]:
df_bike['start_time'] = df_bike.start_time.astype('datetime64[s]')
df_bike['stop_time'] = df_bike.stop_time.astype('datetime64[s]')
df_bike['trip_duration'] = df_bike.apply(lambda row: (row['stop_time'] - row['start_time']).total_seconds(), axis=1)
df_bike['user_type'] = pd.Categorical(df_bike.user_type, ['Subscriber', 'Customer'])
df_bike['start_station_name'] = df_bike.start_station_name.astype('string')
df_bike['end_station_name'] = df_bike.end_station_name.astype('string')

**Note:** We cannot cast the `birth_year` into an integer value as it contains missing data.

We then validate that the datatypes have been defined correctly.

In [31]:
print(df_bike.dtypes)

trip_duration                     float64
start_time                  datetime64[s]
stop_time                   datetime64[s]
start_station_id                    int64
start_station_name         string[python]
start_station_latitude            float64
start_station_longitude           float64
end_station_id                      int64
end_station_name           string[python]
end_station_latitude              float64
end_station_longitude             float64
bike_id                             int64
user_type                        category
birth_year                        float64
gender                              int64
dtype: object


---
### Summary Statistics

We now want to describe the data to see if the summary statistics indicate the data is as we would expect it to be.

In [36]:
df_bike[['trip_duration', 'start_station_latitude', 'start_station_longitude', 'end_station_latitude', 'end_station_longitude', 'birth_year']].describe()

,trip_duration,start_station_latitude,start_station_longitude,end_station_latitude,end_station_longitude,birth_year
count,2.475840e+05,247584.000000,247584.000000,247584.000000,247584.000000,228585.000000
mean,8.861297e+02,40.723121,-74.046438,40.722594,-74.045855,1979.335276
std,3.593810e+04,0.008199,0.011211,0.007958,0.011283,9.596809
min,-3.271000e+03,40.692640,-74.096937,40.692216,-74.096937,1900.000000
25%,2.490000e+02,40.717732,-74.050656,40.716540,-74.050444,1974.000000
50%,3.900000e+02,40.721525,-74.044247,40.721124,-74.043117,1981.000000
75%,6.670000e+02,40.727596,-74.038051,40.727224,-74.036486,1986.000000
max,1.632981e+07,40.752559,-74.032108,40.801343,-73.957390,2000.000000


#### Missing Data


The data dictionary states that if a value of zero is given for gender then the gender is unknown. We can therefore convert zero values into `pd.nan` values so that we can evaluate how much data is missing with `df_bike.isnull().sum()`.

Using a *lambda* function, we convert any value of gender that is a `0` value to `None`.

In [41]:
df_bike['gender'] = df_bike.gender.apply(lambda x: None if x != 1 and x != 2 else x)

We can then observe how much of the data is missing:

In [44]:
print(df_bike.isnull().sum())

trip_duration                  0
start_time                     0
stop_time                      0
start_station_id               0
start_station_name             0
start_station_latitude         0
start_station_longitude        0
end_station_id                 0
end_station_name               0
end_station_latitude           0
end_station_longitude          0
bike_id                        0
user_type                    380
birth_year                 18999
gender                     19901
dtype: int64


#### Observations

1. The `trip_duration` max value is an order of magnitude above data in the 3 percentiles.
1. The min `birth_year` is conveniently `1900`.
1. The `birth_year` has missing data.
1. The `user_type` has missing data.
1. The data dictionary states that the `gender` is unknown when the value is 0.

---
### Cleaning CitiBike Data

From the above observations, we can begin cleaning the data so that it is ready for analysis.

#### Trip Duration

In order to view the outliers, we should inspect the data for the `trip_duration` column in descending order.

In [51]:
print(np.sort(df_bike.trip_duration)[:10])
print(np.sort(df_bike.trip_duration)[-10:])

[-3271. -2484.    61.    61.    61.    61.    61.    61.    61.    61.]
[ 1120971.  1258737.  1532001.  1569766.  1837255.  2071209.  2100552.
  2104124.  4826890. 16329808.]


A trip duration cannot have negative value, meaning that these values are incorrect and should be removed.

##### Removing Negative Trip Durations

In [55]:
df_bike = df_bike[df_bike.trip_duration > 0]

##### Invalid Data

CitiBike state that the longest valid trip duration is 24 hours. Therefore any data that lasts longer than this amount of time is considered invalid. This data should not be removed as there may be a reason for it (such as faulty bikes or improper use) and may be of interest to analysts.

A new column will be introduced named `valid_trip_duration`. This will have a value of `1` if the trip was under `86400` seconds (number of seconds in a day) or `0` otherwise.

In [60]:
df_bike['valid_trip_duration'] = df_bike.trip_duration.apply(lambda t: 1 if t < 86400 else 0).astype('bool')

#### Birth Year Outliers

In order to view our outliers, we should inspect the data for the `birth_year` column in ascending order.

In [64]:
print(np.sort(df_bike.birth_year.values)[:10])

[1900. 1934. 1937. 1937. 1937. 1937. 1940. 1940. 1940. 1941.]


We can see that there is only a single outlier, hence we can remove the data at this value.

In [67]:
df_bike = df_bike[df_bike.birth_year != 1900]

#### Missing Birth Year Data

In [70]:
missing_birth_year = df_bike[df_bike[['birth_year']].isnull().any(axis=1)]

In [72]:
print(missing_birth_year.head(10))

     trip_duration          start_time           stop_time  start_station_id  \
125         3666.0 2016-02-01 09:56:46 2016-02-01 10:57:52              3212   
148         1082.0 2016-02-01 11:43:51 2016-02-01 12:01:53              3183   
154         1101.0 2016-02-01 12:04:35 2016-02-01 12:22:56              3192   
163         1580.0 2016-02-01 12:26:09 2016-02-01 12:52:29              3192   
168         1620.0 2016-02-01 12:53:44 2016-02-01 13:20:44              3186   
174          558.0 2016-02-01 13:40:17 2016-02-01 13:49:35              3183   
190          278.0 2016-02-01 14:51:43 2016-02-01 14:56:21              3187   
202          756.0 2016-02-01 16:30:41 2016-02-01 16:43:17              3209   
208         1020.0 2016-02-01 17:00:36 2016-02-01 17:17:36              3183   
517         3412.0 2016-02-02 10:51:14 2016-02-02 11:48:06              3195   

     start_station_name  start_station_latitude  start_station_longitude  \
125     Christ Hospital               40.73

From inspecting the head we can see that all records which are missing data from the `birth_year` appear to be of `user_type` `Customer`, and also of unknown `gender`. In order to validate that this pattern is consistent, we can calculate the proportion of missing birth years to total customers.

In [75]:
proportion_customers_missing_birth_year = 100 * missing_birth_year.user_type.value_counts() / df_bike.user_type.value_counts()
print(proportion_customers_missing_birth_year)

user_type
Subscriber     1.523222
Customer      99.671413
Name: count, dtype: float64


The above analysis demonstrates that 99.7% of all our customer data do not include the data of birth. In order to investigate why, we should investigate methods of data collection.
For now, we should not remove data where the birth year is missing as otherwise we would lose all our customer data.

We also saw that when the `birth_year` was missing, `gender` also appeared to be missing. To validate this we calculate the proportion of records with missing `gender` when `birth_year` is missing.

In [79]:
print(missing_birth_year.gender.value_counts())

Series([], Name: count, dtype: int64)


All records with a missing `birth_year` are also missing `gender`.

#### User Type Missing Data

We determined previously that if both the `gender` and the `birth_year` value are missing then the `user_type` is likely a `Customer`, else it is a `Subscriber`.

In [84]:
missing_user_type = df_bike[df_bike.user_type.isnull()]

We can see how many records with a missing `user_type` could be a potential customer.

In [87]:
potential_customer_count = missing_user_type[(missing_user_type.gender == 0) | (missing_user_type.birth_year.isnull())].shape[0]
print(potential_customer_count)

0


This means that it is likely that all records that have a missing `user_type` are subscribers, however they may still be customers.

---
#### Resetting the Index

Since we dropped some data, lets reset the index.

In [92]:
df_bike.reset_index(drop=True, inplace=True)
df_bike.bike_id = df_bike.index
df_bike.tail()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,valid_trip_duration
247576,250.0,2016-05-31 23:16:00,2016-05-31 23:20:10,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,247576,Subscriber,1976.0,1.0,True
247577,651.0,2016-05-31 23:18:32,2016-05-31 23:29:23,3209,Brunswick St,40.724176,-74.050656,3211,Newark Ave,40.721525,-74.046305,247577,Subscriber,1986.0,1.0,True
247578,2048.0,2016-05-31 23:25:28,2016-05-31 23:59:36,3199,Newport Pkwy,40.728745,-74.032108,3199,Newport Pkwy,40.728745,-74.032108,247578,Subscriber,1975.0,1.0,True
247579,455.0,2016-05-31 23:31:57,2016-05-31 23:39:32,3220,5 Corners Library,40.734961,-74.059503,3215,Central Ave,40.746730,-74.049251,247579,Subscriber,1964.0,1.0,True
247580,239.0,2016-05-31 23:47:38,2016-05-31 23:51:37,3185,City Hall,40.717732,-74.043845,3211,Newark Ave,40.721525,-74.046305,247580,Subscriber,1993.0,1.0,True


---
---
## Loading Weather Data

In [95]:
df_weather = pd.read_csv('data/newark_airport_2016.csv')

---
### Inspecting the Data

In [98]:
print(df_weather.head(5))

       STATION                                         NAME        DATE  \
0  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-01   
1  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-02   
2  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-03   
3  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-04   
4  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-01-05   

    AWND  PGTM  PRCP  SNOW  SNWD  TAVG  TMAX  TMIN  TSUN  WDF2   WDF5  WSF2  \
0  12.75   NaN   0.0   0.0   0.0    41    43    34   NaN   270  280.0  25.9   
1   9.40   NaN   0.0   0.0   0.0    36    42    30   NaN   260  260.0  21.0   
2  10.29   NaN   0.0   0.0   0.0    37    47    28   NaN   270  250.0  23.9   
3  17.22   NaN   0.0   0.0   0.0    32    35    14   NaN   330  330.0  25.9   
4   9.84   NaN   0.0   0.0   0.0    19    31    10   NaN   360  350.0  25.1   

   WSF5  
0  35.1  
1  25.1  
2  30.0  
3  33.1  
4  31.1  


In [100]:
print(df_weather.tail(5))

         STATION                                         NAME        DATE  \
361  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-12-27   
362  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-12-28   
363  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-12-29   
364  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-12-30   
365  USW00014734  NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US  2016-12-31   

      AWND  PGTM  PRCP  SNOW  SNWD  TAVG  TMAX  TMIN  TSUN  WDF2   WDF5  WSF2  \
361  13.65   NaN  0.01   0.0   0.0    53    62    40   NaN   270  270.0  29.1   
362   8.28   NaN  0.00   0.0   0.0    41    43    31   NaN   330  330.0  19.9   
363   8.05   NaN  0.36   0.0   0.0    38    45    31   NaN   170  150.0  18.1   
364  14.99   NaN  0.00   0.0   0.0    37    42    32   NaN   270  270.0  25.9   
365  12.30   NaN  0.00   0.0   0.0    35    44    29   NaN   200  220.0  21.9   

     WSF5  
361  38.0  
362  25.1  
363  25.1  
36

Upon inspection of the data there are a few observations to be made.
- All `PGTM` values are missing.
- All `PRCP` values are `0.0`.
- All `SNOW` values are `0.0`.
- All `SNWD` values are `0.0`.
- All `TSUN` values are missing.
- The values for the `NAME` column are identical.
- The values for the `STATION` column are identical.

#### Column Data
- AWND: Average Daily Wind Speed (m/s)
- PGTM: Peak Gust Time (HHMM)
- PRCP: Precipitation (mm)
- SNOW: Snowfall (mm)
- SNWD: Snow Depth (mm)
- TAVG: Temperature Average
- TMIN: Minimum Temp
- TSUN: Daily Total Sunshine (minutes)
- WDF2: Direction of Fastest 2 Minute Wind (degrees)
- WDF5: Direction of Fastest 5 Minute Wind (degrees)
- WSF2: Fastest 2 Min Wind Speed (m/s)
- WSF5: Fastest 5 Min Wind Speed (m/s)

If there is significant Snowfall, we would expect Snow Depth. This is something we can investigate.

---
### Renaming Columns

In [106]:
df_weather.columns = [x.lower() for x in df_weather.columns]
df_weather = df_weather.rename(columns={'snow': 'snow_amt', 'snwd': 'snow_depth', 'awnd': 'avg_wind_speed'})

In [108]:
print(df_weather.columns)

Index(['station', 'name', 'date', 'avg_wind_speed', 'pgtm', 'prcp', 'snow_amt',
       'snow_depth', 'tavg', 'tmax', 'tmin', 'tsun', 'wdf2', 'wdf5', 'wsf2',
       'wsf5'],
      dtype='object')


---
#### Inspecting `name` and `station` Columns

As the data showed all the values for the `name` and `station` column as identical we want to inspect the data to see if this is a consistent pattern throughout.

In [112]:
df_weather.station.value_counts()

station
USW00014734    366
Name: count, dtype: int64

In [114]:
df_weather.name.value_counts()

name
NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US    366
Name: count, dtype: int64

The above demonstrates that the same value is present for the `name` and `station` columns throughout the dataset. Therefore we can drop the `name` and `station` columns.

In [117]:
df_weather = df_weather.drop(['station', 'name'], axis=1)

---
#### Missing `tsun` Data

We can now inspect how much of the `tsun` data is missing.

In [121]:
df_weather[df_weather.tsun.isnull() != False].shape[0] / df_weather.shape[0]

1.0

All of the `tsun` data is missing. Keeping this data will lead to database inefficiencies, hence we should remove this data.

In [124]:
df_weather = df_weather.drop('tsun', axis=1)

In [126]:
df_weather.columns

Index(['date', 'avg_wind_speed', 'pgtm', 'prcp', 'snow_amt', 'snow_depth',
       'tavg', 'tmax', 'tmin', 'wdf2', 'wdf5', 'wsf2', 'wsf5'],
      dtype='object')

---
#### Missing `pgtm` Data

We can inspect how much of the `pgtm` data is missing.

In [130]:
df_weather[df_weather.pgtm.isnull() != False].shape[0] / df_weather.shape[0]

1.0

All of the `pgtm` data is missing. Keeping this data will lead to database inefficiences, hence we should remove this data.

In [133]:
df_weather = df_weather.drop('pgtm', axis=1)

In [135]:
df_weather.columns

Index(['date', 'avg_wind_speed', 'prcp', 'snow_amt', 'snow_depth', 'tavg',
       'tmax', 'tmin', 'wdf2', 'wdf5', 'wsf2', 'wsf5'],
      dtype='object')

---
### Columns `wdf2` `wdf5` `wsf2` `wsf5`

These columns are not really relevant as we do not know what time of day they occurred at and so it will be hard to link it to our other data. It is best we drop this data.

In [139]:
df_weather = df_weather.drop(['wdf2', 'wdf5', 'wsf2', 'wsf5'], axis=1)

---
### Checking Column Data Types

In [146]:
print(df_weather.dtypes)

date              datetime64[s]
avg_wind_speed          float64
prcp                    float64
snow_amt                float64
snow_depth              float64
tavg                      int64
tmax                      int64
tmin                      int64
dtype: object


In [144]:
df_weather.date = df_weather.date.astype('datetime64[s]')

In [148]:
print(df_weather.dtypes)

date              datetime64[s]
avg_wind_speed          float64
prcp                    float64
snow_amt                float64
snow_depth              float64
tavg                      int64
tmax                      int64
tmin                      int64
dtype: object


---
#### Inspecting `snow` and `snwd` Data

In [151]:
df_weather_snow = df_weather[(df_weather.snow_amt != 0) | (df_weather.snow_depth != 0)]

In [153]:
df_weather_snow.shape[0]

28

In [155]:
df_weather_snow.head(5)

,date,avg_wind_speed,prcp,snow_amt,snow_depth,tavg,tmax,tmin
16,2016-01-17,9.40,0.07,0.7,0.0,38,42,29
17,2016-01-18,17.22,0.03,0.5,1.2,27,30,18
21,2016-01-22,10.29,0.03,0.3,0.0,26,30,20
22,2016-01-23,22.82,1.81,24.0,7.1,26,27,23
23,2016-01-24,9.40,0.01,0.2,20.1,26,36,17


We can see that on 2016-01-23 there was 7.1mm of snow depth however the next day there was 20.1mm of snow depth despite there only being 0.2mm of snowfall the next day. This may be some error in reporting.

---
#### Snow and Rain Columns

We can add snow and rain columns which allow for convenient analysis on whether it snowed or rained on that day.

In [160]:
df_weather['snow'] = df_weather.snow_amt.apply(lambda x: 1 if x > 0 else 0).astype('bool')
df_weather['rain'] = df_weather.prcp.apply(lambda x: 1 if x > 0 else 0).astype('bool')

In [162]:
print(df_weather.head())

        date  avg_wind_speed  prcp  snow_amt  snow_depth  tavg  tmax  tmin  \
0 2016-01-01           12.75   0.0       0.0         0.0    41    43    34   
1 2016-01-02            9.40   0.0       0.0         0.0    36    42    30   
2 2016-01-03           10.29   0.0       0.0         0.0    37    47    28   
3 2016-01-04           17.22   0.0       0.0         0.0    32    35    14   
4 2016-01-05            9.84   0.0       0.0         0.0    19    31    10   

    snow   rain  
0  False  False  
1  False  False  
2  False  False  
3  False  False  
4  False  False  


---
### Summary Statistics

In [165]:
df_weather[['avg_wind_speed', 'prcp', 'snow_amt', 'snow_depth', 'tavg', 'tmax', 'tmin']].describe()

,avg_wind_speed,prcp,snow_amt,snow_depth,tavg,tmax,tmin
count,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000
mean,9.429973,0.104945,0.098087,0.342623,57.196721,65.991803,48.459016
std,3.748174,0.307496,1.276498,2.078510,17.466981,18.606301,17.135790
min,2.460000,0.000000,0.000000,0.000000,8.000000,18.000000,0.000000
25%,6.765000,0.000000,0.000000,0.000000,43.000000,51.250000,35.000000
50%,8.720000,0.000000,0.000000,0.000000,56.000000,66.000000,47.000000
75%,11.410000,0.030000,0.000000,0.000000,74.000000,83.000000,64.000000
max,22.820000,2.790000,24.000000,20.100000,89.000000,99.000000,80.000000


---
---
## Creating Tables

### Date Dimension Table

As our data frequently uses the date, it makes sense to get all the data and

In [169]:
df_date = pd.DataFrame({"date": pd.date_range('2016-01-01', '2016-12-31')})
df_date["day"] = df_date.date.dt.day_name()
df_date["week"] = df_date.date.dt.isocalendar().week
df_date["quarter"] = df_date.date.dt.quarter
df_date["year"] = df_date.date.dt.year
df_date["weekend"] = df_date.day.apply(lambda x: 1 if x == 'Saturday' or x == 'Sunday' else 0).astype('bool')

In [171]:
df_date['date_key'] = df_date.date.apply(lambda x: int(x.strftime('%Y%m%d').strip('-')))
df_bike['date_key'] = df_bike.start_time.apply(lambda x: int(x.strftime('%Y%m%d').strip('-')))
df_weather['date_key'] = df_weather.date.apply(lambda x: int(x.strftime('%Y%m%d').strip('-')))

In [172]:
print(df_date.dtypes)

date        datetime64[ns]
day                 object
week                UInt32
quarter              int32
year                 int32
weekend               bool
date_key             int64
dtype: object


### Users Table

The rides table will use the bike ID as it's Index, hence no index will be set.

In [177]:
df_users = df_bike[['user_type', 'gender', 'birth_year']].drop_duplicates().reset_index(drop=True)
df_users['id'] = df_users.index
rides = df_bike.merge(df_users, on=['user_type', 'gender', 'birth_year']).sort_values(by='bike_id')
rides = rides.drop(['user_type', 'gender', 'birth_year', 'id'], axis=1)
print(rides.head())

   trip_duration          start_time           stop_time  start_station_id  \
0          361.0 2016-02-01 00:31:18 2016-02-01 00:37:19              3202   
1          297.0 2016-02-01 01:55:05 2016-02-01 02:00:02              3195   
2         1155.0 2016-02-01 02:40:05 2016-02-01 02:59:20              3183   
3         1770.0 2016-02-01 05:11:28 2016-02-01 05:40:58              3214   
4          935.0 2016-02-01 05:48:24 2016-02-01 06:03:59              3203   

  start_station_name  start_station_latitude  start_station_longitude  \
0       Newport PATH               40.727224               -74.033759   
1            Sip Ave               40.730743               -74.063784   
2     Exchange Place               40.716247               -74.033459   
3   Essex Light Rail               40.712774               -74.036486   
4      Hamilton Park               40.727596               -74.044247   

   end_station_id  end_station_name  end_station_latitude  \
0            3203     Hamilton 

### Start Station Table

In [180]:
rides.columns

Index(['trip_duration', 'start_time', 'stop_time', 'start_station_id',
       'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bike_id',
       'valid_trip_duration', 'date_key'],
      dtype='object')

In [182]:
start_station = rides[['start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude']]
end_station = rides[['end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude']]

start_station = start_station.rename(columns={'start_station_id': 'id', 'start_station_name': 'name', 'start_station_latitude': 'latitude', 'start_station_longitude': 'longitude'})
end_station = end_station.rename(columns={'end_station_id': 'id', 'end_station_name': 'name', 'end_station_latitude': 'latitude', 'end_station_longitude': 'longitude'})

station = pd.concat([start_station, end_station]).drop_duplicates()
print(station.head())

     id              name   latitude  longitude
0  3202      Newport PATH  40.727224 -74.033759
1  3195           Sip Ave  40.730743 -74.063784
2  3183    Exchange Place  40.716247 -74.033459
3  3214  Essex Light Rail  40.712774 -74.036486
4  3203     Hamilton Park  40.727596 -74.044247


The rides table already contains the `start_station_id` and the `end_station_id`, hence no merge is required. Any column relating to a station that is not the index can be removed. 

In [185]:
rides = rides.drop(['start_station_name', 'start_station_latitude', 'start_station_longitude', 'end_station_name', 'end_station_latitude', 'end_station_longitude'], axis=1)

In [187]:
print(rides.head())

   trip_duration          start_time           stop_time  start_station_id  \
0          361.0 2016-02-01 00:31:18 2016-02-01 00:37:19              3202   
1          297.0 2016-02-01 01:55:05 2016-02-01 02:00:02              3195   
2         1155.0 2016-02-01 02:40:05 2016-02-01 02:59:20              3183   
3         1770.0 2016-02-01 05:11:28 2016-02-01 05:40:58              3214   
4          935.0 2016-02-01 05:48:24 2016-02-01 06:03:59              3203   

   end_station_id  bike_id  valid_trip_duration  date_key  
0            3203        0                 True  20160201  
1            3194        1                 True  20160201  
2            3210        2                 True  20160201  
3            3203        3                 True  20160201  
4            3214        4                 True  20160201  


In [189]:
rides.dtypes

trip_duration                float64
start_time             datetime64[s]
stop_time              datetime64[s]
start_station_id               int64
end_station_id                 int64
bike_id                        int64
valid_trip_duration             bool
date_key                       int64
dtype: object

### Weather Table

In [192]:
df_weather['id'] = df_weather.index

In [194]:
print(df_weather.head())

        date  avg_wind_speed  prcp  snow_amt  snow_depth  tavg  tmax  tmin  \
0 2016-01-01           12.75   0.0       0.0         0.0    41    43    34   
1 2016-01-02            9.40   0.0       0.0         0.0    36    42    30   
2 2016-01-03           10.29   0.0       0.0         0.0    37    47    28   
3 2016-01-04           17.22   0.0       0.0         0.0    32    35    14   
4 2016-01-05            9.84   0.0       0.0         0.0    19    31    10   

    snow   rain  date_key  id  
0  False  False  20160101   0  
1  False  False  20160102   1  
2  False  False  20160103   2  
3  False  False  20160104   3  
4  False  False  20160105   4  


In [196]:
print(df_weather.dtypes)

date              datetime64[s]
avg_wind_speed          float64
prcp                    float64
snow_amt                float64
snow_depth              float64
tavg                      int64
tmax                      int64
tmin                      int64
snow                       bool
rain                       bool
date_key                  int64
id                        int64
dtype: object


The weather table can remain as is as there are no logical groupings to split out.

---
#### Connect to a PostGres Database

In [200]:
import sqlalchemy as sal
from sqlalchemy import create_engine

In [202]:
print(station.head())

     id              name   latitude  longitude
0  3202      Newport PATH  40.727224 -74.033759
1  3195           Sip Ave  40.730743 -74.063784
2  3183    Exchange Place  40.716247 -74.033459
3  3214  Essex Light Rail  40.712774 -74.036486
4  3203     Hamilton Park  40.727596 -74.044247


In [204]:
engine = sal.create_engine('postgresql://arbergllogjani:@127.0.0.1:10000/codecademydataengineering')

In [206]:
connection = engine.connect()

In [210]:
df_date.to_sql('date_dimension', connection, if_exists='append', index=False, chunksize=10000)
station.to_sql('station', connection, if_exists='append', index=False, chunksize=10000)
df_users.to_sql('users', connection, if_exists='append', index=False, chunksize=10000)
df_weather.to_sql('weather', connection, if_exists='append', index=False, chunksize=10000)
rides.to_sql('rides', connection, if_exists='append', index=False, chunksize=10000)
print('Acknowledge Data Uploaded')

Acknowledge Data Uploaded
